In [407]:
import numpy as np
import matplotlib.pyplot as plt

In [542]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)


In [528]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [529]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [530]:
X_st = X.copy()
X_st[1, :] = calc_std_feat(X[1, :])
X_st[2, :] = calc_std_feat(X[2, :])
X_st[3, :] = calc_std_feat(X[3, :])

### 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log
Произведем замену нуля и единицы на значения близкие к ним соответственно, на величину допустимой ошибки. Добавим параметр accuracy(величину допустимой ошибки)к функции calc_logloss

In [531]:
def calc_logloss(y, y_pred,accuracy = 1e-10):    
    
    y_pred = y_pred*1.0
    y_pred[y_pred==1.0] = 1-accuracy #Заменяет все единицы, если таковые есть, числом близким к 1  
    y_pred[y_pred==0] = accuracy   #Заменяет все нули, если таковые есть, числом определенным параметром accuracy
    
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальны

In [535]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
       # if i % (iterations / 10) == 0:
        #    print(i, W, err)
    return W,err

__Ошибка уменьшается с ростом параметра iterations при фиксированном параметре alpha=1e-1__

In [603]:
alpha = [0.1**x for x in np.arange(1,7)]
iterations = [10**x for x in np.arange(1,8)]

def min_logloss(iterations,alpha):
    resalt = []
    for iteration in iterations:
        for item_alpha in alpha:            
            value = eval_model(X_st, y, iterations=iteration, alpha=item_alpha)
            resalt.append([value[1],iteration,item_alpha,value[0]])    
    return np.array(resalt)

In [604]:
min_err_logloss = min_logloss(iterations,alpha)
i = np.argmin(min_err_logloss[:,0:3], axis=0)[0]
result = min_err_logloss[i:i+1,:]
result

array([[0.0019925103606355894, 10000000, 0.1,
        array([ 34.94384443, -57.21094237, -90.18859581, 179.199862  ])]],
      dtype=object)

In [563]:
min_err_logloss = min_logloss(iterations,alpha)
i = np.argmin(min_err_logloss[:,0:3], axis=0)[0]
result = min_err_logloss[i:i+1,:]
result

array([[0.018903424327598818, 1000000, 0.1,
        array([ 18.48339152, -31.66669169, -45.72519691,  93.39039439])]],
      dtype=object)

In [565]:
min_err_logloss[:,3:]

array([[array([ 0.44971291, -0.37392863,  0.43417109,  1.41927966])],
       [array([ 0.49299873, -0.16323078,  0.62466872,  1.51095461])],
       [array([ 0.49635347, -0.14077377,  0.64537095,  1.52180621])],
       [array([ 0.49667819, -0.13851537,  0.64745662,  1.52290733])],
       [array([ 0.49671056, -0.13828941,  0.64766535,  1.5230176 ])],
       [array([ 0.49671379, -0.13826681,  0.64768622,  1.52302863])],
       [array([ 0.09138926, -1.16094315, -0.1252748 ,  1.70944249])],
       [array([ 0.44889713, -0.37252874,  0.4358295 ,  1.42095128])],
       [array([ 0.49298692, -0.16321668,  0.6246859 ,  1.51097235])],
       [array([ 0.49635335, -0.14077362,  0.64537112,  1.52180639])],
       [array([ 0.49667819, -0.13851537,  0.64745663,  1.52290733])],
       [array([ 0.49671056, -0.13828941,  0.64766535,  1.5230176 ])],
       [array([ 0.37731566, -2.73985947, -1.18135067,  4.58774123])],
       [array([ 0.09249613, -1.16047276, -0.12476714,  1.70921775])],
       [array([ 0.44

### 3.Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [575]:
def calc_pred_proba(W,X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W,X)))
    return y_pred_proba 

In [592]:
W = result[0][3]
calc_pred_proba(W ,X_st)

array([8.85614391e-02, 6.26364745e-10, 1.00000000e+00, 7.80133447e-06,
       9.93567196e-01, 4.10345796e-09, 1.00000000e+00, 1.63083231e-03,
       9.15569109e-01, 1.00000000e+00])

### 4.Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [593]:
def calc_pred(W,X):
    y_pred_proba = calc_pred_proba(W,X)
    y_pred = []
    for i in y_pred_proba:
        if (i > 0.5): 
            y_pred.append(1)
        elif (i <= 0.5):
            y_pred.append(0)
    return np.array(y_pred) 

In [596]:
y_pred = calc_pred(W ,X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [647]:
def accurac(y, y_pred):    
    return np.mean(np.abs(y == y_pred))
                   
accurac(y,y_pred)                  

1.0

__Построим матрицу ошибок__

In [599]:
def error_matrix(y, y_pred):
    error_matrix = np.zeros((2,2))
    for key, item_y in enumerate(y):
        if item_y == 1 and y_pred[key] == 1:
            error_matrix[0,0] += 1
        elif item_y == 1 and y_pred[key] == 0:
            error_matrix[1,0] += 1
        elif item_y == 0 and y_pred[key] == 1:
            error_matrix[0,1] += 1
        elif item_y == 0 and y_pred[key] == 0:
            error_matrix[1,1] += 1
            
        
    return error_matrix

error_matrix_res = error_matrix(y, y_pred)

In [600]:
y_test = np.array([0,0,0,1,1,0,1,0,1,0,1,0])
y_pred_test = np.array([1,0,1,0,1,0,1,0,0,1,1,0])

error_matrix_res = error_matrix(y_test, y_pred_test)
error_matrix_res

array([[3., 3.],
       [2., 4.]])

__Определим точность и полноту__

In [601]:
def 𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛(error_matrix):
    return error_matrix[0,0]/(error_matrix[0,0] + error_matrix[0,1])

def 𝑟𝑒𝑐𝑎𝑙𝑙(error_matrix):
    return error_matrix[0,0]/(error_matrix[0,0] + error_matrix[1,0])


𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛_ = 𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛(error_matrix_res)
𝑟𝑒𝑐𝑎𝑙𝑙_ = 𝑟𝑒𝑐𝑎𝑙𝑙(error_matrix_res)
print(f"𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛: {𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛_}")
print(f"𝑟𝑒𝑐𝑎𝑙𝑙: {𝑟𝑒𝑐𝑎𝑙𝑙_}")

𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛: 0.5
𝑟𝑒𝑐𝑎𝑙𝑙: 0.6


__Определим F1 score__

In [602]:
def score_f_n(error_matrix,n:int=1):
    f_score = (1+n**2)*𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛(error_matrix) * 𝑟𝑒𝑐𝑎𝑙𝑙(error_matrix)/(n**2 * 𝑝𝑟𝑒𝑐𝑖𝑠𝑖𝑜𝑛(error_matrix) + 𝑟𝑒𝑐𝑎𝑙𝑙(error_matrix))
    return f_score
score_f_n(error_matrix_res)    

0.5454545454545454

### 6. Могла ли модель переобучиться? Почему?

Одним из признаков, что произошло переобучение модели, получение больших по модулю весов при признаках.
При изменении параметра  iterations функции eval_model в сторону увеличения уменьшается ошибка предсказания целевой переменной, но одновременно с этим растут веса при признаках, что говорит о том, что с  увеличением iterations, модель способна переобучиться. При iterations=100000, модель не переобучилась. Другим признаком переобучения является значительный перекос значений метрик между обучающей и тестовой выборках. В нашей модели этого перекоса нет. Вывод: модель не переобучилась.

In [609]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [625]:
X_st = X.copy()
X_st[1, :] = calc_std_feat(X[1, :])
X_st[2, :] = calc_std_feat(X[2, :])
X_st[3, :] = calc_std_feat(X[3, :])
X_st = X_st.T

In [626]:
# перемешивание датасета
np.random.seed(12)
shuffle_index = np.random.permutation(X_st.shape[0])
X_shuffled, y_shuffled = X_st[shuffle_index], y[shuffle_index]

# разбивка на обучающую и тестовую выборки
train_proportion = 0.7
train_test_cut = int(len(X_st) * train_proportion)

X_train, X_test, y_train, y_test = \
    X_shuffled[:train_test_cut], \
    X_shuffled[train_test_cut:], \
    y_shuffled[:train_test_cut], \
    y_shuffled[train_test_cut:]
    
print("Размер массива признаков обучающей выборки", X_train.shape)
print("Размер массива признаков тестовой выборки", X_test.shape)
print("Размер массива ответов для обучающей выборки", y_train.shape)
print("Размер массива ответов для тестовой выборки", y_test.shape)

Размер массива признаков обучающей выборки (7, 4)
Размер массива признаков тестовой выборки (3, 4)
Размер массива ответов для обучающей выборки (7,)
Размер массива ответов для тестовой выборки (3,)


In [654]:
n_iterations = 100000
eta = 0.1

w, losses = eval_model(X_train.T, y_train.T, n_iterations, eta)

y_predicted_test = calc_pred(w, X_test.T)
y_predicted_train = calc_pred(w, X_train.T)

# В качестве меры точности возьмем долю правильных ответов
train_accuracy = accurac(y_train,y_predicted_train)*100 
test_accuracy = accurac(y_test,y_predicted_test)*100


print(f"Итоговый вектор весов w: {w}")
print(f"Точность на обучающей выборке: {train_accuracy:.3f}")
print(f"Точность на тестовой выборке: {test_accuracy:.3f}")

Итоговый вектор весов w: [  8.49898796 -16.21191028 -18.69294049  41.30996939]
Точность на обучающей выборке: 100.000
Точность на тестовой выборке: 100.000


In [655]:
error_matrix(y_test,y_predicted_test)

array([[2., 0.],
       [0., 1.]])